# Homework Module 1

## Q1. Running Elastic Search

What's the `version.build_hash` value?

A: 42f05b9372a9a4a470db3b52817899b99a76ee73

In [ ]:
response = {
    "name": "c25bf73d9244",
    "cluster_name": "docker-cluster",
    "cluster_uuid": "hALgKmI0QFSax3AcYqm1CQ",
    "version": {
        "number": "8.4.3",
        "build_flavor": "default",
        "build_type": "docker",
        "build_hash": "42f05b9372a9a4a470db3b52817899b99a76ee73",
        "build_date": "2022-10-04T07:17:24.662462378Z",
        "build_snapshot": "false",
        "lucene_version": "9.3.0",
        "minimum_wire_compatibility_version": "7.17.0",
        "minimum_index_compatibility_version": "7.0.0"
    },
    "tagline": "You Know, for Search"
}

## Q2. Which function do you use for adding your data to elastic?

A: `index`

In [8]:
import requests
from elasticsearch import Elasticsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
response = requests.get(docs_url)
raw_docs = response.json()

documents = []

for item in raw_docs:
    course_name = item["course"]

    for doc in item["documents"]:
        doc["course"] = course_name
        documents.append(doc)

elastic_search = Elasticsearch('http://localhost:9200/')
body_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}
index_name = "course-questions-homework"

elastic_search.indices.create(index=index_name, body=body_settings)

for document in documents:
    elastic_search.index(index=index_name, document=document)


## Q3. What's the score for the top ranking result?

A: 24.50

In [26]:
question = "How do execute a command on a Kubernetes pod?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

response = elastic_search.search(index = index_name, body= search_query)

scores = []

for item in response["hits"]["hits"]:
    score = item["_score"]
    scores.append(score)

max(scores)

31.973522

## Q4. What's the 3rd question returned by the search engine?

A: 'How do I copy files from a different folder into docker container’s working directory?'

In [27]:
question = "How do copy a file to a Docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = elastic_search.search(index = index_name, body= search_query)

result_docs = []
for hit in response["hits"]["hits"]:
    result_docs.append(hit["_source"])

questions = []
for item in response["hits"]["hits"]:
    question = item["_source"]["question"]
    questions.append(question)

questions[2]

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. What's the length of the resulting prompt?

A: 2446

In [31]:
question =  "How do I execute a command in a running docker container?"

prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT:
        {context}
    """.strip()

context = ""

for doc in result_docs:
    context += f"section: {doc["section"]}\nquestion: {doc["question"]}\nanswer: {doc["text"]}\n\n"

prompt = prompt_template.format(
    question = question,
    context = context
)

len(prompt)

2474

## Q6. How many tokens does our prompt have?

A: 420

In [38]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
decoding_token = encoding.decode_single_token_bytes(4165)

print(decoding_token)
len(tokens)


b' course'


545

## Bonus: Generating the answer

In [39]:
%load_ext dotenv
%dotenv

In [43]:
from openai import OpenAI

openai_client = OpenAI()

response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'To execute a command in a running Docker container, you can use the following steps:\n\n1. List all running containers to find the container ID of the one you want to execute a command in:\n\n   ```bash\n   docker ps\n   ```\n\n2. Use the `docker exec` command to run a command inside the running container. For example, to start a bash session:\n\n   ```bash\n   docker exec -it <container-id> bash\n   ```\n\nReplace `<container-id>` with the container ID obtained from the `docker ps` command.'

## Bonus: Calculating the costs

In [50]:
input_tokens_price = 0.005
output_tokens_price = 0.015

input_tokens = 150
output_tokens = 250

input_tokens_amount = (input_tokens_price * input_tokens) / 1000
output_tokens_amount = (output_tokens_price * output_tokens) / 1000
cost_per_request = input_tokens_amount + output_tokens_amount

total_requests = 1000

cost_total_requests = total_requests * cost_per_request

print("Total cost of 1000 requests: ", str(cost_total_requests), "$")

Total cost of 1000 requests:  4.5 $
